In [2]:
import data

class Tablet:
    def __init__(self, instructions):
        self.registers = {}
        self.pos = 0
        self.parse(instructions)
        
    def parse(self, instructions):
        instructions = [instruction.split(' ') for instruction in instructions.split('\n')]
        n = 0
        while True:
            inst = instructions[self.pos]
            res = self.process(inst)

            if res:
                break
                            
            n += 1
            
    def process(self, line):
        op = line[0]
        if len(line) > 1: 
            reg = line[1]

        reg_val = self.get(reg)

        if len(line) > 2:
            val = line[2]
            if is_numeric(val):
                val = int(val)
            else:
                val = self.get(val)

        if op == 'snd': 
            self.played = self.get(reg)
        elif op == 'set':
            self.set(reg, val)
        elif op == 'add':
            self.set(reg, reg_val + val)
        elif op == 'mul':
            self.set(reg, reg_val * val)
        elif op == 'mod':
            self.set(reg, reg_val % val)
        elif op == 'rcv':
            if reg_val != 0:
                return self.played
        elif op == 'jgz':
            if reg_val > 0:
                self.pos += val 
                return 
        self.pos += 1
    
    def get(self, register):
        if register not in self.registers:
            self.registers[register] = 0
        return self.registers[register]
    
    def set(self, register, val):
        self.registers[register] = val

def is_numeric(string):
    try:
        int(string)
        return True
    except ValueError:
        return False
        
        
t = Tablet(data.test_data)
assert t.played == 4
t = Tablet(data.data)
print(t.played)

3423


# Part 2

In [31]:
class Tablet:
    def __init__(self, instructions, p):
        self.registers = {}
        self.pos = 0
        self.parse(instructions)
        self.set('p', p)
        self.p = p
        self.messages = []
        self.sent = 0
        self.processed = 0
        self.done = False
        self.played = None
        
    def parse(self, instructions):
        self.instructions = [instruction.split(' ') for instruction in instructions.split('\n')]
                    
    def partner(self, program):
        self.partner = program
            
    def process(self):
        if self.pos < 0 or self.pos >= len(self.instructions):
            return 'end'
        
        if self.done:
            return
        
        self.processed += 1
        
        line = self.instructions[self.pos]
        print(self.p, ' '.join(line))
        
        op = line[0]
        
        # Get the argument and it's value 
        arg1, arg1_val = self.get(line[1])

        if len(line) > 2:
            arg2, arg2_val = self.get(line[2])
            
        if op == 'snd':
            self.partner.messages.append(arg1_val)
            self.sent +=1 
            self.played = arg1_val
#             print('%i sent %i' % (self.p, arg1_val))
        elif op == 'set':
            self.set(arg1, arg2_val)
        elif op == 'add':
            self.set(arg1, arg1_val + arg2_val)
        elif op == 'mul':
            self.set(arg1, arg1_val * arg2_val)
        elif op == 'mod':
            self.set(arg1, arg1_val % arg2_val)
        elif op == 'rcv':
            if len(self.messages) == 0:
                return 'end'
            else:
                self.set(arg1, self.messages[0])
                self.messages = self.messages[1:]
        elif op == 'jgz':
            if arg1_val > 0:
                self.pos += arg2_val
                return 
        self.pos += 1
        print(self)

        
        
    def get(self, register):
        # Part 2 allows this to not be a register 
        if is_numeric(register):
            return None, int(register)
        
        if register not in self.registers:
            self.registers[register] = 0
            
        return register, self.registers[register]
    
    def set(self, register, val):
        self.registers[register] = val
        
        
    def __str__(self):
        s = '%i - {' % self.p
        for k, v in self.registers.items():
            s += '%s: %i, ' % (k, v)
        s = s[:-2] + '}'
        return s

        
def parse_if_int(arg):
    if is_numeric(arg):
        return int(arg)
    else:
        return arg
        
# t0 = Tablet(data.test_data_2, 0)
# t1 = Tablet(data.test_data_2, 1)
# t0.partner(t1)
# t1.partner(t0)
# i = 0
# while True:
#     end1 = t0.process()
#     end2 = t1.process()
#     if end1 == 'end' and end1 =='end':
#         break
#     i += 1
# print(i, t0.sent, t1.sent)

# REAL DEAL
t0 = Tablet(data.data, 0)
t1 = Tablet(data.data, 1)
t0.partner(t1)
t1.partner(t0)
i = 0
while True:
#     print(i)
    end1 = t0.process()
    end2 = t1.process()
    if end1 == 'end' and end1 =='end':
        break
    i += 1
print(i, t0.sent, t1.sent)


0 set i 31
0 - {i: 31, p: 0}
1 set i 31
1 - {i: 31, p: 1}
0 set a 1
0 - {a: 1, i: 31, p: 0}
1 set a 1
1 - {a: 1, i: 31, p: 1}
0 mul p 17
0 - {a: 1, i: 31, p: 0}
1 mul p 17
1 - {a: 1, i: 31, p: 17}
0 jgz p p
0 - {a: 1, i: 31, p: 0}
1 jgz p p
0 mul a 2
0 - {a: 2, i: 31, p: 0}
1 jgz a 3
0 add i -1
0 - {a: 2, i: 30, p: 0}
1 set f 0
1 - {a: 1, i: 31, p: 17, f: 0}
0 jgz i -2
1 set i 126
1 - {a: 1, i: 126, p: 17, f: 0}
0 mul a 2
0 - {a: 4, i: 30, p: 0}
1 rcv a
0 add i -1
0 - {a: 4, i: 29, p: 0}
1 rcv a
0 jgz i -2
1 rcv a
0 mul a 2
0 - {a: 8, i: 29, p: 0}
1 rcv a
0 add i -1
0 - {a: 8, i: 28, p: 0}
1 rcv a
0 jgz i -2
1 rcv a
0 mul a 2
0 - {a: 16, i: 28, p: 0}
1 rcv a
0 add i -1
0 - {a: 16, i: 27, p: 0}
1 rcv a
0 jgz i -2
1 rcv a
0 mul a 2
0 - {a: 32, i: 27, p: 0}
1 rcv a
0 add i -1
0 - {a: 32, i: 26, p: 0}
1 rcv a
0 jgz i -2
1 rcv a
0 mul a 2
0 - {a: 64, i: 26, p: 0}
1 rcv a
0 add i -1
0 - {a: 64, i: 25, p: 0}
1 rcv a
0 jgz i -2
1 rcv a
0 mul a 2
0 - {a: 128, i: 25, p: 0}
1 rcv a
0 add i -1
0 -

0 mul p 8505
0 - {b: 4027, a: 2147483647, i: 82, p: 904711099635}
1 add p b
1 - {b: 4027, a: 450, i: 83, p: 3577, f: 1}
0 mod p a
0 - {b: 4027, a: 2147483647, i: 82, p: 620484248}
1 jgz p 4
0 mul p 129749
0 - {b: 4027, a: 2147483647, i: 82, p: 80507210693752}
1 snd b
1 - {b: 4027, a: 450, i: 83, p: 3577, f: 1}
0 add p 12345
0 - {b: 4027, a: 2147483647, i: 82, p: 80507210706097}
1 set f 1
1 - {b: 4027, a: 450, i: 83, p: 3577, f: 1}
0 mod p a
0 - {b: 4027, a: 2147483647, i: 82, p: 196263714}
1 add i -1
1 - {b: 4027, a: 450, i: 82, p: 3577, f: 1}
0 set b p
0 - {b: 196263714, a: 2147483647, i: 82, p: 196263714}
1 jgz i -11
0 mod b 10000
0 - {b: 3714, a: 2147483647, i: 82, p: 196263714}
1 rcv b
0 snd b
0 - {b: 3714, a: 2147483647, i: 82, p: 196263714}
1 rcv b
1 - {b: 3714, a: 450, i: 82, p: 3577, f: 1}
0 add i -1
0 - {b: 3714, a: 2147483647, i: 81, p: 196263714}
1 set p a
1 - {b: 3714, a: 450, i: 82, p: 450, f: 1}
0 jgz i -9
1 mul p -1
1 - {b: 3714, a: 450, i: 82, p: -450, f: 1}
0 mul p 850

1 - {b: 105, a: 105, i: 38, p: -290, f: 1}
0 mod b 10000
0 - {b: 2294, a: 2147483647, i: 38, p: 651092294}
1 jgz i -11
0 snd b
0 - {b: 2294, a: 2147483647, i: 38, p: 651092294}
1 rcv b
1 - {b: 2294, a: 105, i: 38, p: -290, f: 1}
0 add i -1
0 - {b: 2294, a: 2147483647, i: 37, p: 651092294}
1 set p a
1 - {b: 2294, a: 105, i: 38, p: 105, f: 1}
0 jgz i -9
1 mul p -1
1 - {b: 2294, a: 105, i: 38, p: -105, f: 1}
0 mul p 8505
0 - {b: 2294, a: 2147483647, i: 37, p: 5537539960470}
1 add p b
1 - {b: 2294, a: 105, i: 38, p: 2189, f: 1}
0 mod p a
0 - {b: 2294, a: 2147483647, i: 37, p: 1327118504}
1 jgz p 4
0 mul p 129749
0 - {b: 2294, a: 2147483647, i: 37, p: 172192298775496}
1 snd b
1 - {b: 2294, a: 105, i: 38, p: 2189, f: 1}
0 add p 12345
0 - {b: 2294, a: 2147483647, i: 37, p: 172192298787841}
1 set f 1
1 - {b: 2294, a: 105, i: 38, p: 2189, f: 1}
0 mod p a
0 - {b: 2294, a: 2147483647, i: 37, p: 617520440}
1 add i -1
1 - {b: 2294, a: 105, i: 37, p: 2189, f: 1}
0 set b p
0 - {b: 617520440, a: 21474

0 mul p -1
0 - {b: 2145, a: 2853, i: 119, p: -2853, f: 1}
1 set a b
1 - {b: 3260, a: 3260, i: 121, p: -1130, f: 1}
0 add p b
0 - {b: 2145, a: 2853, i: 119, p: -708, f: 1}
1 jgz 1 3
0 jgz p 4
0 - {b: 2145, a: 2853, i: 119, p: -708, f: 1}
1 add i -1
1 - {b: 3260, a: 3260, i: 120, p: -1130, f: 1}
0 snd a
0 - {b: 2145, a: 2853, i: 119, p: -708, f: 1}
1 jgz i -11
0 set a b
0 - {b: 2145, a: 2145, i: 119, p: -708, f: 1}
1 rcv b
1 - {b: 2853, a: 3260, i: 120, p: -1130, f: 1}
0 jgz 1 3
1 set p a
1 - {b: 2853, a: 3260, i: 120, p: 3260, f: 1}
0 add i -1
0 - {b: 2145, a: 2145, i: 118, p: -708, f: 1}
1 mul p -1
1 - {b: 2853, a: 3260, i: 120, p: -3260, f: 1}
0 jgz i -11
1 add p b
1 - {b: 2853, a: 3260, i: 120, p: -407, f: 1}
0 rcv b
0 - {b: 1278, a: 2145, i: 118, p: -708, f: 1}
1 jgz p 4
1 - {b: 2853, a: 3260, i: 120, p: -407, f: 1}
0 set p a
0 - {b: 1278, a: 2145, i: 118, p: 2145, f: 1}
1 snd a
1 - {b: 2853, a: 3260, i: 120, p: -407, f: 1}
0 mul p -1
0 - {b: 1278, a: 2145, i: 118, p: -2145, f: 1}
1

1 - {b: 576, a: 577, i: 75, p: -1, f: 1}
0 jgz p 4
1 jgz p 4
1 - {b: 576, a: 577, i: 75, p: -1, f: 1}
0 snd b
0 - {b: 9586, a: 450, i: 73, p: 9136, f: 1}
1 snd a
1 - {b: 576, a: 577, i: 75, p: -1, f: 1}
0 set f 1
0 - {b: 9586, a: 450, i: 73, p: 9136, f: 1}
1 set a b
1 - {b: 576, a: 576, i: 75, p: -1, f: 1}
0 add i -1
0 - {b: 9586, a: 450, i: 72, p: 9136, f: 1}
1 jgz 1 3
0 jgz i -11
1 add i -1
1 - {b: 576, a: 576, i: 74, p: -1, f: 1}
0 rcv b
0 - {b: 2674, a: 450, i: 72, p: 9136, f: 1}
1 jgz i -11
0 set p a
0 - {b: 2674, a: 450, i: 72, p: 450, f: 1}
1 rcv b
1 - {b: 9586, a: 576, i: 74, p: -1, f: 1}
0 mul p -1
0 - {b: 2674, a: 450, i: 72, p: -450, f: 1}
1 set p a
1 - {b: 9586, a: 576, i: 74, p: 576, f: 1}
0 add p b
0 - {b: 2674, a: 450, i: 72, p: 2224, f: 1}
1 mul p -1
1 - {b: 9586, a: 576, i: 74, p: -576, f: 1}
0 jgz p 4
1 add p b
1 - {b: 9586, a: 576, i: 74, p: 9010, f: 1}
0 snd b
0 - {b: 2674, a: 450, i: 72, p: 2224, f: 1}
1 jgz p 4
0 set f 1
0 - {b: 2674, a: 450, i: 72, p: 2224, f: 1}

...

0 jgz i -11
1 rcv a
1 - {b: 105, a: 8332, i: 126, p: -290, f: 0}
0 rcv b
0 - {b: 6004, a: 7656, i: 124, p: -582, f: 0}
1 rcv b
1 - {b: 8238, a: 8332, i: 126, p: -290, f: 0}
0 set p a
0 - {b: 6004, a: 7656, i: 124, p: 7656, f: 0}
1 set p a
1 - {b: 8238, a: 8332, i: 126, p: 8332, f: 0}
0 mul p -1
0 - {b: 6004, a: 7656, i: 124, p: -7656, f: 0}
1 mul p -1
1 - {b: 8238, a: 8332, i: 126, p: -8332, f: 0}
0 add p b
0 - {b: 6004, a: 7656, i: 124, p: -1652, f: 0}
1 add p b
1 - {b: 8238, a: 8332, i: 126, p: -94, f: 0}
0 jgz p 4
0 - {b: 6004, a: 7656, i: 124, p: -1652, f: 0}
1 jgz p 4
1 - {b: 8238, a: 8332, i: 126, p: -94, f: 0}
0 snd a
0 - {b: 6004, a: 7656, i: 124, p: -1652, f: 0}
1 snd a
1 - {b: 8238, a: 8332, i: 126, p: -94, f: 0}
0 set a b
0 - {b: 6004, a: 6004, i: 124, p: -1652, f: 0}
1 set a b
1 - {b: 8238, a: 8238, i: 126, p: -94, f: 0}
0 jgz 1 3
1 jgz 1 3
0 add i -1
0 - {b: 6004, a: 6004, i: 123, p: -1652, f: 0}
1 add i -1
1 - {b: 8238, a: 8238, i: 125, p: -94, f: 0}
0 jgz i -11
1 jgz i -

0 - {b: 743, a: 619, i: 78, p: 124, f: 1}
1 set a b
1 - {b: 693, a: 693, i: 81, p: -23, f: 1}
0 jgz i -11
1 jgz 1 3
0 rcv b
0 - {b: 1857, a: 619, i: 78, p: 124, f: 1}
1 add i -1
1 - {b: 693, a: 693, i: 80, p: -23, f: 1}
0 set p a
0 - {b: 1857, a: 619, i: 78, p: 619, f: 1}
1 jgz i -11
0 mul p -1
0 - {b: 1857, a: 619, i: 78, p: -619, f: 1}
1 rcv b
1 - {b: 743, a: 693, i: 80, p: -23, f: 1}
0 add p b
0 - {b: 1857, a: 619, i: 78, p: 1238, f: 1}
1 set p a
1 - {b: 743, a: 693, i: 80, p: 693, f: 1}
0 jgz p 4
1 mul p -1
1 - {b: 743, a: 693, i: 80, p: -693, f: 1}
0 snd b
0 - {b: 1857, a: 619, i: 78, p: 1238, f: 1}
1 add p b
1 - {b: 743, a: 693, i: 80, p: 50, f: 1}
0 set f 1
0 - {b: 1857, a: 619, i: 78, p: 1238, f: 1}
1 jgz p 4
0 add i -1
0 - {b: 1857, a: 619, i: 77, p: 1238, f: 1}
1 snd b
1 - {b: 743, a: 693, i: 80, p: 50, f: 1}
0 jgz i -11
1 set f 1
1 - {b: 743, a: 693, i: 80, p: 50, f: 1}
0 rcv b
0 - {b: 8850, a: 619, i: 77, p: 1238, f: 1}
1 add i -1
1 - {b: 743, a: 693, i: 79, p: 50, f: 1}
0 

0 set a b
0 - {b: 576, a: 576, i: 49, p: -1, f: 1}
1 jgz p 4
0 jgz 1 3
1 snd b
1 - {b: 4638, a: 619, i: 51, p: 4019, f: 1}
0 add i -1
0 - {b: 576, a: 576, i: 48, p: -1, f: 1}
1 set f 1
1 - {b: 4638, a: 619, i: 51, p: 4019, f: 1}
0 jgz i -11
1 add i -1
1 - {b: 4638, a: 619, i: 50, p: 4019, f: 1}
0 rcv b
0 - {b: 7782, a: 576, i: 48, p: -1, f: 1}
1 jgz i -11
0 set p a
0 - {b: 7782, a: 576, i: 48, p: 576, f: 1}
1 rcv b
1 - {b: 577, a: 619, i: 50, p: 4019, f: 1}
0 mul p -1
0 - {b: 7782, a: 576, i: 48, p: -576, f: 1}
1 set p a
1 - {b: 577, a: 619, i: 50, p: 619, f: 1}
0 add p b
0 - {b: 7782, a: 576, i: 48, p: 7206, f: 1}
1 mul p -1
1 - {b: 577, a: 619, i: 50, p: -619, f: 1}
0 jgz p 4
1 add p b
1 - {b: 577, a: 619, i: 50, p: -42, f: 1}
0 snd b
0 - {b: 7782, a: 576, i: 48, p: 7206, f: 1}
1 jgz p 4
1 - {b: 577, a: 619, i: 50, p: -42, f: 1}
0 set f 1
0 - {b: 7782, a: 576, i: 48, p: 7206, f: 1}
1 snd a
1 - {b: 577, a: 619, i: 50, p: -42, f: 1}
0 add i -1
0 - {b: 7782, a: 576, i: 47, p: 7206, f: 1

0 mul p -1
0 - {b: 1952, a: 440, i: 19, p: -440, f: 1}
1 rcv b
1 - {b: 2802, a: 448, i: 21, p: 2072, f: 1}
0 add p b
0 - {b: 1952, a: 440, i: 19, p: 1512, f: 1}
1 set p a
1 - {b: 2802, a: 448, i: 21, p: 448, f: 1}
0 jgz p 4
1 mul p -1
1 - {b: 2802, a: 448, i: 21, p: -448, f: 1}
0 snd b
0 - {b: 1952, a: 440, i: 19, p: 1512, f: 1}
1 add p b
1 - {b: 2802, a: 448, i: 21, p: 2354, f: 1}
0 set f 1
0 - {b: 1952, a: 440, i: 19, p: 1512, f: 1}
1 jgz p 4
0 add i -1
0 - {b: 1952, a: 440, i: 18, p: 1512, f: 1}
1 snd b
1 - {b: 2802, a: 448, i: 21, p: 2354, f: 1}
0 jgz i -11
1 set f 1
1 - {b: 2802, a: 448, i: 21, p: 2354, f: 1}
0 rcv b
0 - {b: 1650, a: 440, i: 18, p: 1512, f: 1}
1 add i -1
1 - {b: 2802, a: 448, i: 20, p: 2354, f: 1}
0 set p a
0 - {b: 1650, a: 440, i: 18, p: 440, f: 1}
1 jgz i -11
0 mul p -1
0 - {b: 1650, a: 440, i: 18, p: -440, f: 1}
1 rcv b
1 - {b: 1952, a: 448, i: 20, p: 2354, f: 1}
0 add p b
0 - {b: 1650, a: 440, i: 18, p: 1210, f: 1}
1 set p a
1 - {b: 1952, a: 448, i: 20, p: 448

KeyboardInterrupt: 

In [9]:
t1.pos, len(t1.instructions)

(7, 7)

In [10]:
t1.instructions[7]

IndexError: list index out of range

In [11]:
t1.messages

[0]